In [1]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,RvcProcessorData
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf

2023-08-13 22:03:20 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-13 22:03:20 | INFO | faiss.loader | Loading faiss with AVX2 support.
2023-08-13 22:03:20 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


cuda:0
 Key with which the item will be registered {'device': 'cuda:0'}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60, 'x_max': 65}


/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_index" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
from speakers.common.registry import registry
registry.get_path("rvc_library_root")

'/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc'

In [3]:
 

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
_,rvc = load_preprocess(config=config.get('preprocess'))
rvc

{'preprocess': {'vits_processor': {'name': 'blip_image_eval', 'image_size': 224}, 'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'rvc/model/hubert_base.pt', 'rvc_config_file': '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/rvc.yaml'}}}
{'name': 'rvc_speakers', 'hubert_model_path': 'rvc/model/hubert_base.pt', 'rvc_config_file': '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/rvc.yaml'}


2023-08-13 22:03:20 | INFO | fairseq.tasks.hubert_pretraining | current directory is /media/gpt4-pdf-chatbot-langchain/RVC-Speakers
2023-08-13 22:03:20 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-08-13 22:03:20 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout

/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/rmvpe.pt
Loading model: yiqing
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/yiqing/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: bob
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/bob/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: arianagrande
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/arianagrande/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: syz
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/syz/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: lulu
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RV

In [4]:
import edge_tts
import asyncio
import nest_asyncio

nest_asyncio.apply()
# Edge TTS speakers
tts_speakers_list = asyncio.get_event_loop().run_until_complete(edge_tts.list_voices())  # noqa

async def edge_tts_vc_func(
        rvc,input_text, model_index, tts_speaker, pitch_adjust, f0_method, 
        feat_ratio, filter_radius, rms_mix_rate, resample_option,
        rate: str, volume: str
):
    if input_text is None:
        return (None, 'Please provide TTS text.')

    if tts_speaker is None:
        return (None, 'Please select TTS speaker.')

    if model_index is None:
        return (None, 'Please select a model.')

    speaker = tts_speakers_list[tts_speaker]['ShortName']
    (tts_np, tts_sr) = await util.call_edge_tts_config(speaker_name=speaker, text=input_text,rate=rate, volume=volume)
 
    # 将 NumPy 数组转换为 Python 列表
    audio_samples_list = tts_np.tolist()
    data = RvcProcessorData( 
        sample_rate=tts_sr, 
        audio_samples= audio_samples_list ,
        model_index= model_index,
        f0_up_key= pitch_adjust,
        f0_method= f0_method,
        index_rate= feat_ratio,
        filter_radius= filter_radius,
        rms_mix_rate= rms_mix_rate,
        resample_sr= resample_option 
    )
    
    return rvc(data)


In [5]:

loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc.loaded_models
]
loaded_models

['Bilibili - 一清清清',
 'ALL - Bob Sponge',
 'ALL - Ariana Grande',
 'ALL - Stefanie Sun',
 '露露 - 露露',
 '毛毛 - 毛毛']

In [6]:

model_index = loaded_models.index('露露 - 露露')
model_index

4

In [7]:

tts_speaker = [
    '%s (%s)' % (
        s['FriendlyName'],
        s['Gender'], 
    )
    for s in tts_speakers_list
] 

In [8]:
"""

 'Microsoft HiuGaai Online (Natural) - Chinese (Cantonese Traditional) (Female)',
 'Microsoft HiuMaan Online (Natural) - Chinese (Hong Kong) (Female)',
 'Microsoft WanLung Online (Natural) - Chinese (Hong Kong) (Male)',
 'Microsoft Xiaoxiao Online (Natural) - Chinese (Mainland) (Female)',
 'Microsoft Xiaoyi Online (Natural) - Chinese (Mainland) (Female)',
 'Microsoft Yunjian Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Yunxi Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Yunxia Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Yunyang Online (Natural) - Chinese (Mainland) (Male)',
 'Microsoft Xiaobei Online (Natural) - Chinese (Northeastern Mandarin) (Female)',
 'Microsoft HsiaoChen Online (Natural) - Chinese (Taiwan) (Female)',
 'Microsoft YunJhe Online (Natural) - Chinese (Taiwan) (Male)',
 'Microsoft HsiaoYu Online (Natural) - Chinese (Taiwanese Mandarin) (Female)',
 'Microsoft Xiaoni Online (Natural) - Chinese (Zhongyuan Mandarin Shaanxi) (Female)',
"""
tts_speaker_index = tts_speaker.index('Microsoft Xiaoxiao Online (Natural) - Chinese (Mainland) (Female)')
tts_speaker_index

55

In [9]:
value="封建五爵，井田刑狱"
# 变调
pitch_adjust=0
f0_method =['pm', 'harvest']

feat_ratio=0.6
filter_radius=3
rms_mix_rate=1
resample_rate=0

rate='-20%'
volume='-50%' 
out_sr, output_audio = await edge_tts_vc_func(rvc,value,model_index,tts_speaker_index,pitch_adjust,f0_method[1],
                                feat_ratio, filter_radius, rms_mix_rate, resample_rate,
                                   rate=rate, volume=volume)



0
npy: 0.4836466312408447s, f0: 2.204667806625366s, infer: 0.4648118019104004s


In [10]:
from scipy.io.wavfile import write as write_wav

# save audio to disk
write_wav(f"test.wav", out_sr, output_audio) 

# play text in notebook
Audio(output_audio, rate=out_sr)

In [11]:
json_data = '''
[
    {
        "title": "菩萨蛮 其一",
        "paragraphs": [
            "小山重叠金明灭，鬓云欲度香腮雪。",
            "懒起画蛾眉，弄妆梳洗迟。",
            "照花前后镜，花面交 相映。",
            "新帖绣罗襦，双双金鹧鸪。"
        ],
        "author": "温庭筠",
        "rhythmic": "菩萨蛮",
        "notes": [
            "1.小山--写女子的隔夜残妆。小山：女子画眉的式样之一。小山重叠：眉晕褪色。金：额黄，在额上涂黄色。金明灭：褪色的额黄明暗不匀。",
            "2.鬓云欲度--鬓发撩乱如云，低垂下来。香腮雪：洁白如雪的香腮。",
            "3.照花--对镜簪花。用前镜、后镜对照以瞻顾后影。",
            "4.双双--罗襦上用金线绣的成双的鹧鸪鸟。反衬自身孤独。"
        ]
    }
]
'''

In [13]:
import json
import pyaudio
import numpy as np
import asyncio


#出错Could not import the PyAudio C module 'pyaudio._portaudio'.
#运行下面命令
#conda install -c anaconda portaudio    

async def play_audio(audio_data, rate):
    loop = asyncio.get_event_loop()
    p = pyaudio.PyAudio()

    def callback(in_data, frame_count, time_info, status):
        nonlocal audio_data
        data = audio_data[:frame_count]
        audio_data = audio_data[frame_count:]
        return (data.tobytes(), pyaudio.paContinue)

    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=rate,
                    output=True,
                    stream_callback=callback)

    stream.start_stream()
    while stream.is_active():
        await asyncio.sleep(0.1)

    stream.stop_stream()
    stream.close()
    p.terminate()


In [14]:
import time


async def text_to_speech(data):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=22050,
                    output=True)
    for item in data:
        
        timestamp = int(time.time())
        for key, value in item.items():
            
            if isinstance(value, str): 
                out_sr, output_audio = await edge_tts_vc_func(rvc,value,model_index,tts_speaker_index,pitch_adjust,f0_method[1],
                                        feat_ratio, filter_radius, rms_mix_rate, resample_rate,
                                           rate=rate, volume=volume)
                 
                # save audio to disk
                write_wav(f"wav/{timestamp}_{key}.wav", out_sr, output_audio) 
                await play_audio(output_audio,out_sr)
            elif isinstance(value, list):
                for i, para in enumerate(value): 
                    out_sr, output_audio = await edge_tts_vc_func(rvc,para,model_index,tts_speaker_index,pitch_adjust,f0_method[1],
                                        feat_ratio, filter_radius, rms_mix_rate, resample_rate,
                                           rate=rate, volume=volume)
                    
                    # save audio to disk
                    write_wav(f"wav/{timestamp}_{key}_{i}.wav", out_sr, output_audio) 
                    await play_audio(output_audio,out_sr)

In [15]:

import os

'''
获取输入文件夹内的所有wav文件，并返回文件名全称列表
'''
def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        print(files)
        for file in files:
            if os.path.splitext(file)[1] == '.json':
                filename=os.path.join(root, file)
                L.append(filename)
    return L 

def read_text_file(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return None
    except Exception as e:
        print("An error occurred while reading the file:", e)
        return None

    
json_dir="/media/checkpoint/chinese-poetry/曹操诗集/"#文件夹
json_texts=file_name(json_dir)#获取文件夹内的所有文件
print(json_texts)
for text in json_texts:
    text_content = read_text_file(text)
    data = json.loads(text_content)
    await text_to_speech(data)

    

['README.md', 'caocao.json']
['/media/checkpoint/chinese-poetry/曹操诗集/caocao.json']


ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib dlmisc.c:337:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_jack.so (/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/libjack.so.0))
ALSA lib dlmisc.c:337:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_jack.so (/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python

0
npy: 0.026175737380981445s, f0: 1.5087964534759521s, infer: 0.12660980224609375s
0
npy: 0.027698755264282227s, f0: 1.904057502746582s, infer: 0.1345057487487793s
0
npy: 0.027729272842407227s, f0: 2.0786726474761963s, infer: 0.1341266632080078s
0
npy: 0.02692246437072754s, f0: 2.269228219985962s, infer: 0.13306760787963867s
0
npy: 0.02840876579284668s, f0: 1.9938387870788574s, infer: 0.13101673126220703s
0
npy: 0.010556459426879883s, f0: 2.110210418701172s, infer: 0.06036639213562012s
0
npy: 0.009449243545532227s, f0: 0.0006155967712402344s, infer: 0.05995821952819824s
0
npy: 0.00973963737487793s, f0: 2.2952041625976562s, infer: 0.061743736267089844s
0
npy: 0.027848482131958008s, f0: 2.164949417114258s, infer: 0.13628554344177246s
0
npy: 0.024564743041992188s, f0: 2.335561752319336s, infer: 0.12783360481262207s
0
npy: 0.009603261947631836s, f0: 2.203042507171631s, infer: 0.06015729904174805s
0
npy: 0.025003910064697266s, f0: 2.387863874435425s, infer: 0.13512349128723145s
0
npy: 0.009

ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred


0
npy: 0.025951862335205078s, f0: 1.892559289932251s, infer: 0.13726139068603516s
0
npy: 0.009842634201049805s, f0: 2.1217215061187744s, infer: 0.06062459945678711s
0
npy: 0.009742975234985352s, f0: 2.047975540161133s, infer: 0.05996823310852051s
0
npy: 0.026919126510620117s, f0: 2.470463991165161s, infer: 0.15707778930664062s


...

0
npy: 0.009210348129272461s, f0: 1.4455831050872803s, infer: 0.05308818817138672s
0
npy: 0.02868342399597168s, f0: 2.470334768295288s, infer: 0.14270448684692383s
0
npy: 0.009795665740966797s, f0: 2.324723482131958s, infer: 0.06287765502929688s
0
npy: 0.025472402572631836s, f0: 2.2392425537109375s, infer: 0.1290292739868164s


ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred


0
npy: 0.010000944137573242s, f0: 2.511906147003174s, infer: 0.06307554244995117s
0
npy: 0.009445667266845703s, f0: 2.3594279289245605s, infer: 0.06168198585510254s
0
npy: 0.025979042053222656s, f0: 2.171166181564331s, infer: 0.13599419593811035s
0
npy: 0.02597784996032715s, f0: 2.127387523651123s, infer: 0.1332252025604248s
0
npy: 0.026192188262939453s, f0: 2.2623462677001953s, infer: 0.12932229042053223s
0
npy: 0.0247652530670166s, f0: 2.3335046768188477s, infer: 0.13855838775634766s
0
npy: 0.01100015640258789s, f0: 2.318206310272217s, infer: 0.06197357177734375s
0
npy: 0.026753664016723633s, f0: 2.2214722633361816s, infer: 0.13843321800231934s
0
npy: 0.01077413558959961s, f0: 2.227250337600708s, infer: 0.06270241737365723s
0
npy: 0.02542281150817871s, f0: 1.9334652423858643s, infer: 0.13039016723632812s
0
npy: 0.026537179946899414s, f0: 2.067049264907837s, infer: 0.1309983730316162s
0
npy: 0.0268094539642334s, f0: 2.2476322650909424s, infer: 0.1381666660308838s
0
npy: 0.024522542953

ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8675:(snd_pcm_recover) underrun occurred


CancelledError: 